<a href="https://colab.research.google.com/github/MandavaNagasreeKalpana/Generative_Ai_analytics_vidya/blob/main/Water_qua_dl_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import pandas as pd
!pip install torch --upgrade

In [21]:
data=pd.read_csv("/content/drive/MyDrive/water_quality.csv")
data.head()

,Well_ID,State,District,Block,Village,Latitude,Longitude,Year,pH,EC,...,NO3,TH,Ca,Mg,Na,K,F,TDS,WQI,Water Quality Classification
0,W232200071580001,Gujarat,Ahmedabad,Mandal,Dalod,NaN,NaN,2020,8.20,16640.0,...,26.00,1451.0,152.0,260.0,3535.0,45.0,1.00,11149,4361.44080,Unsuitable for Drinking
1,W254029084355301,Himachal Pradesh,Solan,Nallagarh,JAGATPUR,31.1594,76.678500,2019,8.44,299.0,...,2.70,84.0,17.0,10.0,39.0,2.4,0.20,262,85.80466,Good
2,W193530074180001,Maharashtra,Ahmednagar,SANGAMNER,Kokangaon,74.3000,19.591667,2022,7.90,1315.0,...,18.20,465.0,80.2,64.4,88.3,1.6,0.53,372,280.04130,Very Poor yet Drinkable
3,W231620072033001,Gujarat,Ahmedabad,Viramgam,Endla,NaN,NaN,2020,7.40,715.0,...,0.23,280.0,56.0,34.0,47.0,11.0,0.46,479,195.11649,Poor
4,W251908084361501,Himachal Pradesh,Solan,Nallagarh,BARUNA,31.1540,76.638400,2019,8.40,270.0,...,10.00,116.0,10.0,18.0,23.0,1.0,0.12,262,81.77860,Good


In [22]:
data.isnull().sum()

,0
Well_ID,3785
State,0
District,0
Block,1119
Village,1
Latitude,389
Longitude,390
Year,0
pH,0
EC,0


Removing null values

In [23]:
if 'Well_ID' in data.columns:
    data = data.drop(columns=['Well_ID'])

data['Block'] = data['Block'].fillna(data['Block'].mode()[0])
data['Village'] = data['Village'].fillna(data['Village'].mode()[0])

data['Latitude'] = data['Latitude'].fillna(data['Latitude'].median())
data['Longitude'] = data['Longitude'].fillna(data['Longitude'].median())

display(data.isnull().sum())

,0
State,0
District,0
Block,0
Village,0
Latitude,0
Longitude,0
Year,0
pH,0
EC,0
CO3,0


Regression

In [24]:
y = data['WQI']
X = data.drop(columns=['WQI', 'State', 'District', 'Block', 'Village', 'Water Quality Classification', 'Year'])

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("\nFirst 5 rows of X:")
display(X.head())
print("\nFirst 5 values of y:")
display(y.head())

Shape of X: (19029, 16)
Shape of y: (19029,)

First 5 rows of X:


,Latitude,Longitude,pH,EC,CO3,HCO3,Cl,SO4,NO3,TH,Ca,Mg,Na,K,F,TDS
0,24.178415,76.175000,8.20,16640.0,0.0,1257.0,5176.0,822.0,26.00,1451.0,152.0,260.0,3535.0,45.0,1.00,11149
1,31.159400,76.678500,8.44,299.0,43.0,87.0,21.0,0.0,2.70,84.0,17.0,10.0,39.0,2.4,0.20,262
2,74.300000,19.591667,7.90,1315.0,0.0,518.7,120.5,61.6,18.20,465.0,80.2,64.4,88.3,1.6,0.53,372
3,24.178415,76.175000,7.40,715.0,0.0,354.0,50.0,18.0,0.23,280.0,56.0,34.0,47.0,11.0,0.46,479
4,31.154000,76.638400,8.40,270.0,43.0,87.0,14.0,0.0,10.00,116.0,10.0,18.0,23.0,1.0,0.12,262



First 5 values of y:


,WQI
0,4361.44080
1,85.80466
2,280.04130
3,195.11649
4,81.77860


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (15223, 16)
Shape of X_test: (3806, 16)
Shape of y_train: (15223,)
Shape of y_test: (3806,)


In [26]:
import torch

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

print("Shape of X_train_tensor:", X_train_tensor.shape)
print("Type of X_train_tensor:", X_train_tensor.dtype)
print("\nShape of y_train_tensor:", y_train_tensor.shape)
print("Type of y_train_tensor:", y_train_tensor.dtype)

print("\nShape of X_test_tensor:", X_test_tensor.shape)
print("Type of X_test_tensor:", X_test_tensor.dtype)
print("\nShape of y_test_tensor:", y_test_tensor.shape)
print("Type of y_test_tensor:", y_test_tensor.dtype)

Shape of X_train_tensor: torch.Size([15223, 16])
Type of X_train_tensor: torch.float32

Shape of y_train_tensor: torch.Size([15223, 1])
Type of y_train_tensor: torch.float32

Shape of X_test_tensor: torch.Size([3806, 16])
Type of X_test_tensor: torch.float32

Shape of y_test_tensor: torch.Size([3806, 1])
Type of y_test_tensor: torch.float32


In [27]:
import torch.nn as nn

class RegressionNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        return x

input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1

model = RegressionNN(input_size, hidden_size, output_size)

print(model)
print(f"Input size: {input_size}")
print(f"Output size: {output_size}")

RegressionNN(
  (fc1): Linear(in_features=16, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)
Input size: 16
Output size: 1


In [28]:
import torch.optim as optim
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Loss function (criterion):", criterion)
print("Optimizer:", optimizer)

Loss function (criterion): MSELoss()
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


In [29]:
num_epochs = 100
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    y_pred_train = model(X_train_tensor)

    loss = criterion(y_pred_train, y_train_tensor)

    loss.backward()
    optimizer.step()

    train_losses.append(loss.item())

    model.eval()
    with torch.no_grad():
        y_pred_test = model(X_test_tensor)
        test_loss = criterion(y_pred_test, y_test_tensor)
        test_losses.append(test_loss.item())

    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}')

print('\nTraining complete.')
print(f'Final Train Loss: {train_losses[-1]:.4f}')
print(f'Final Test Loss: {test_losses[-1]:.4f}')

Epoch [1/100], Train Loss: 565170368.0000, Test Loss: 132048.5000
Epoch [10/100], Train Loss: 80568.9922, Test Loss: 75458.0078
Epoch [20/100], Train Loss: 30554.0820, Test Loss: 26315.9023
Epoch [30/100], Train Loss: 7348.3696, Test Loss: 5622.0557
Epoch [40/100], Train Loss: 2849.6985, Test Loss: 2835.1357
Epoch [50/100], Train Loss: 3332.6223, Test Loss: 3625.9116
Epoch [60/100], Train Loss: 2856.6436, Test Loss: 2952.5969
Epoch [70/100], Train Loss: 2304.3843, Test Loss: 2246.9624
Epoch [80/100], Train Loss: 2123.4119, Test Loss: 1990.3611
Epoch [90/100], Train Loss: 1981.0125, Test Loss: 1846.7102
Epoch [100/100], Train Loss: 1821.7808, Test Loss: 1728.4862

Training complete.
Final Train Loss: 1821.7808
Final Test Loss: 1728.4862


In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

model.eval()

with torch.no_grad():
    y_pred_test_tensor = model(X_test_tensor)

y_test_np = y_test_tensor.numpy()
y_pred_test_np = y_pred_test_tensor.numpy()

mae = mean_absolute_error(y_test_np, y_pred_test_np)
mse = mean_squared_error(y_test_np, y_pred_test_np)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_np, y_pred_test_np)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared (R2 Score): {r2:.4f}")

Mean Absolute Error (MAE): 18.9383
Mean Squared Error (MSE): 1728.4863
Root Mean Squared Error (RMSE): 41.5751
R-squared (R2 Score): 0.9836


Classification

In [31]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

y = data['Water Quality Classification']
X = data.drop(columns=['WQI', 'State', 'District', 'Block', 'Village', 'Water Quality Classification', 'Year'])

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y = pd.Series(y_encoded, name='Water Quality Classification_encoded')

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("\nFirst 5 rows of X:")
display(X.head())
print("\nFirst 5 values of y:")
display(y.head())

Shape of X: (19029, 16)
Shape of y: (19029,)

First 5 rows of X:


,Latitude,Longitude,pH,EC,CO3,HCO3,Cl,SO4,NO3,TH,Ca,Mg,Na,K,F,TDS
0,24.178415,76.175000,8.20,16640.0,0.0,1257.0,5176.0,822.0,26.00,1451.0,152.0,260.0,3535.0,45.0,1.00,11149
1,31.159400,76.678500,8.44,299.0,43.0,87.0,21.0,0.0,2.70,84.0,17.0,10.0,39.0,2.4,0.20,262
2,74.300000,19.591667,7.90,1315.0,0.0,518.7,120.5,61.6,18.20,465.0,80.2,64.4,88.3,1.6,0.53,372
3,24.178415,76.175000,7.40,715.0,0.0,354.0,50.0,18.0,0.23,280.0,56.0,34.0,47.0,11.0,0.46,479
4,31.154000,76.638400,8.40,270.0,43.0,87.0,14.0,0.0,10.00,116.0,10.0,18.0,23.0,1.0,0.12,262



First 5 values of y:


,Water Quality Classification_encoded
0,3
1,1
2,4
3,2
4,1


In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (15223, 16)
Shape of X_test: (3806, 16)
Shape of y_train: (15223,)
Shape of y_test: (3806,)


In [33]:
import torch

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

print("Shape of X_train_tensor:", X_train_tensor.shape)
print("Type of X_train_tensor:", X_train_tensor.dtype)
print("\nShape of y_train_tensor:", y_train_tensor.shape)
print("Type of y_train_tensor:", y_train_tensor.dtype)

print("\nShape of X_test_tensor:", X_test_tensor.shape)
print("Type of X_test_tensor:", X_test_tensor.dtype)
print("\nShape of y_test_tensor:", y_test_tensor.shape)
print("Type of y_test_tensor:", y_test_tensor.dtype)

Shape of X_train_tensor: torch.Size([15223, 16])
Type of X_train_tensor: torch.float32

Shape of y_train_tensor: torch.Size([15223])
Type of y_train_tensor: torch.int64

Shape of X_test_tensor: torch.Size([3806, 16])
Type of X_test_tensor: torch.float32

Shape of y_test_tensor: torch.Size([3806])
Type of y_test_tensor: torch.int64


In [34]:
import torch.nn as nn

num_classes = len(y.unique())

class ClassificationNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = num_classes

model = ClassificationNN(input_size, hidden_size, output_size)

print(model)
print(f"Input size: {input_size}")
print(f"Output size: {output_size}")

ClassificationNN(
  (fc1): Linear(in_features=16, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=5, bias=True)
)
Input size: 16
Output size: 5


In [35]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Loss function (criterion):", criterion)
print("Optimizer:", optimizer)

Loss function (criterion): CrossEntropyLoss()
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


In [37]:
num_epochs = 100
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    y_pred_train = model(X_train_tensor)

    loss = criterion(y_pred_train, y_train_tensor)

    loss.backward()
    optimizer.step()

    train_losses.append(loss.item())

    model.eval()
    with torch.no_grad():
        y_pred_test = model(X_test_tensor)
        test_loss = criterion(y_pred_test, y_test_tensor)
        test_losses.append(test_loss.item())

    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}')

print('\nTraining complete.')
print(f'Final Train Loss: {train_losses[-1]:.4f}')
print(f'Final Test Loss: {test_losses[-1]:.4f}')

Epoch [1/100], Train Loss: 2.6810, Test Loss: 2.1174
Epoch [10/100], Train Loss: 2.5184, Test Loss: 1.9457
Epoch [20/100], Train Loss: 2.3588, Test Loss: 1.7864
Epoch [30/100], Train Loss: 2.2140, Test Loss: 1.6440
Epoch [40/100], Train Loss: 2.0820, Test Loss: 1.5243
Epoch [50/100], Train Loss: 1.9680, Test Loss: 1.4180
Epoch [60/100], Train Loss: 1.8677, Test Loss: 1.3231
Epoch [70/100], Train Loss: 1.7780, Test Loss: 1.2374
Epoch [80/100], Train Loss: 1.6991, Test Loss: 1.1532
Epoch [90/100], Train Loss: 1.6279, Test Loss: 1.0721
Epoch [100/100], Train Loss: 1.5628, Test Loss: 1.0002

Training complete.
Final Train Loss: 1.5628
Final Test Loss: 1.0002


In [39]:
from sklearn.metrics import classification_report, accuracy_score

model.eval()
with torch.no_grad():
    y_pred_test_tensor = model(X_test_tensor)
    y_pred_test_np = y_pred_test_tensor.argmax(dim=1).numpy()  # For multi-class

y_test_np = y_test_tensor.numpy().flatten()

print(classification_report(y_test_np, y_pred_test_np))
print(f"Accuracy: {accuracy_score(y_test_np, y_pred_test_np):.4f}")

              precision    recall  f1-score   support

           0       0.79      0.81      0.80       136
           1       0.58      0.36      0.44       317
           2       0.62      0.59      0.61      1066
           3       0.88      0.89      0.89      1370
           4       0.50      0.58      0.54       917

    accuracy                           0.69      3806
   macro avg       0.68      0.65      0.66      3806
weighted avg       0.69      0.69      0.69      3806

Accuracy: 0.6871
